In [1]:
import pandas as pd
import functions as fnc
import variables as var
import seaborn as sns
#import plot_features_num_regression as function

In [2]:
df = pd.read_csv("titanic.csv")
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [3]:
df.corr()

ValueError: could not convert string to float: 'male'

In [ ]:
# get_features_num_regresion
# Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que 
# debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta 
# pero con alta cardinalidad, además de un argumento 'umbral_corr', de tipo float que debe estar entre 0 y 1 y una variable 
# float "pvalue" cuyo valor debe ser por defecto "None".
# La función debe devolver una lista con las columnas numéricas del dataframe cuya correlación con la columna designada por "target_col" 
# sea superior en valor absoluto al valor dado por "umbral_corr". Además si la variable "pvalue" es distinta de None, sólo devolvera 
# las columnas numéricas cuya correlación supere el valor indicado y además supere el test de hipótesis con significación mayor o igual a 1-pvalue.
# La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir 
# hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla 
# la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.
#-----------------------------------------------------------------

# Esta función recibe un dataframe, una argumento "target_col" con valor por defecto "", una lista de strings ("columns") 
# cuyo valor por defecto es la lista vacía, un valor de correlación ("umbral_corr", con valor 0 por defecto) y 
# un argumento ("pvalue") con valor "None" por defecto.

# Si la lista no está vacía, la función pintará una pairplot del dataframe considerando la columna designada por "target_col" 
# y aquellas incluidas en "column" que cumplan que su correlación con "target_col" es superior en valor absoluto a "umbral_corr", 
# y que, en el caso de ser pvalue diferente de "None", además cumplan el test de correlación para el nivel 1-pvalue de 
# significación estadística. La función devolverá los valores de "columns" que cumplan con las condiciones anteriores. 

# EXTRA: Se valorará adicionalmente el hecho de que si la lista de columnas a pintar es grande se pinten varios pairplot con un 
# máximo de cinco columnas en cada pairplot (siendo siempre una de ellas la indicada por "target_col")


# OK: Si la lista está vacía, entonces la función igualará "columns" a las variables numéricas del dataframe y se comportará como
#  se describe en el párrafo anterior.

# OK: De igual manera que en la función descrita anteriormente deberá hacer un check de los valores de entrada y comportarse como
#  se describe en el último párrafo de la función `get_features_num_regresion`

def plot_features_num_regression(dataframe, target_col="", columns=[], umbral_corr=0, pvalue=None):
    """
        <Descripción>

        Argumentos:
            > dataframe: Dataframe con los datos
            > target_col: Columna target a analizar
            > columns: Columnas con las que buscar la correlación con la columna 'target'. En caso de no especificar
                        nada, se revisrán las variables numéricas que hay en el dataframe
            > umbral_corr: Umbral a partir del cual una columna se va a comparar con el target
            > pvalue:

        Retorna:
        <return>

    """
    # Comprobamos si los parámetros son correcttos
    numeric_types = [var.TIPO_NUM_CONTINUA, var.TIPO_NUM_DISCRETA]
    if not fnc.is_valid_params(dataframe, target_col, columns, numeric_types, numeric_types):
        return None
    
    final_columns = columns
    # Si no se pasa parámetro 'columns', extraemos todas las columnas numéricas
    if len(final_columns) == 0:
        df_types = fnc.tipifica_variables(df, var.UMBRAL_CATEGORIA, var.UMBRAL_CONTINUA)
        final_columns = df_types[df_types[var.COLUMN_TIPO].isin(numeric_types)][var.COLUMN_NOMBRE].to_list()
    
    # Borramos la columna target de la lista, en el caso de que exista
    if target_col in final_columns:
        final_columns.remove(target_col)

    if len(final_columns) == 0:
        print("No se han especificado columnas en el parámetro 'columns' y el set de datos no contiene ninguna columna numérica (diferente al target)")
        return None

    #print(df[[target_col] +  final_columns].corr(numeric_only=True)[target_col])

    df_corr_matrix = dataframe[final_columns + [target_col]].corr(numeric_only=True)    
    corr_columns = df_corr_matrix.loc[df_corr_matrix[target_col] >= umbral_corr][target_col].index.to_list()    

    # Borramos la columna target de la lista, en el caso de que exista
    if target_col in corr_columns:
        corr_columns.remove(target_col)

    if len(corr_columns) == 0:
        print("No se han encontrado columnas de correlación con los criterios especificados")
        return None
    else:
        sns.set_style = var.SNS_STYLE
        sns.pairplot(dataframe[[target_col] +  corr_columns])

    return corr_columns
    

columns = plot_features_num_regression(df, "", umbral_corr=1)
columns


La columna '' no existe en el dataframe
